In [ ]:
!pip install datasets transformers accelerate torch scikit-learn matplotlib tensorboard codecarbon TensorFlow pillow==9.5.0 tf-keras


In [ ]:
import os
import time
import csv
import random
import numpy as np
from datasets import load_dataset
from pathlib import Path
import tempfile
import torch
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    ToTensor,
    Resize,
)
from codecarbon import EmissionsTracker
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_recall_fscore_support, accuracy_score
from transformers import EvalPrediction

In [ ]:
################## SETINGS OF THE CODE ################
do_train = True
do_eval = True
do_predict = False

name_of_experiment = "CollabA100"  # this will be the main folder name

# experiments to make
validation_dataset_percentage = 0.1 # when the datase have no validation set then i will consider that is this apount fom the train set

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
################## Paths Setup ################

# Path to save the rsults
workspace_dir = Path("/content")
experiment_dir = workspace_dir/ "experiment" / name_of_experiment
experiment_dir.mkdir(parents=True, exist_ok=True)
cache_dir = workspace_dir/  "database"
temp_dir = workspace_dir/  "tmp"

# Set environment variables to use the custom directories
os.environ["TMPDIR"] = str(temp_dir)
os.environ["TEMP"] = str(temp_dir)
os.environ["TMP"] = str(temp_dir)
os.environ["HF_HOME"] = str(cache_dir)  # Hugging Face cache directory
os.environ["TRANSFORMERS_CACHE"] = str(cache_dir)  # Transformers cache directory

# Use tempfile to set a custom base directory for temporary files
tempfile.tempdir = str(temp_dir)


In [ ]:
##################  Dataset Preparation ################
# Load the dataset with the specified cache directory
dataset = load_dataset("emre570/breastcancer-ultrasound-images", cache_dir=cache_dir)

train_val_split = dataset["train"].train_test_split(test_size=validation_dataset_percentage)

# Merge train and test sets
from datasets import DatasetDict

dataset = DatasetDict(
    {"train": train_val_split["train"], "validation": train_val_split["test"], "test": dataset["test"]}
)

# my datasets, Train, Validate, Test
train_ds = dataset["train"]
val_ds = dataset["validation"]
test_ds = dataset["test"]



# 1. Label Mapping: We convert between label IDs and their corresponding names, useful for model training and evaluation.
id2label = {id: label for id, label in enumerate(train_ds.features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

# 2. Image Processing: Then, we utilize the ViTImageProcessor to standardize input image sizes and applies normalization specific to the pretrained model. Also, will define different transformations for training, validation, and testing to improve model generalization using torchvision.
model_name = "google/vit-large-patch16-224" # this is the model for this finetunning
processor = ViTImageProcessor.from_pretrained(model_name)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)

train_transforms = Compose([
    RandomResizedCrop(size),
    RandomHorizontalFlip(),
    ToTensor(),
    normalize,
])

val_transforms = Compose([
    Resize(size),
    CenterCrop(size),
    ToTensor(),
    normalize,
])

test_transforms = Compose([
    Resize(size),
    CenterCrop(size),
    ToTensor(),
    normalize,
])

# Create transform functions
def apply_train_transforms(examples):
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples

def apply_val_transforms(examples):
    examples["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples

def apply_test_transforms(examples):
    examples["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["image"]]
    return examples

# Apply transform functions to each set . all pixel values have been tranformed to tansors at that point
train_ds.set_transform(apply_train_transforms)
val_ds.set_transform(apply_val_transforms)
test_ds.set_transform(apply_test_transforms)


# Data Loading: Set up a custom collate function to properly batch images and labels, and create a DataLoader for efficient loading and batching during model training.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dl = DataLoader(train_ds, collate_fn=collate_fn, batch_size=4)
val_dl = DataLoader(val_ds, collate_fn=collate_fn, batch_size=4)

# Batch Preparation: Retrieve and display the shape of data in a sample batch to verify correct processing and readiness for model input.
batch = next(iter(train_dl))
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/624 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/156 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

pixel_values torch.Size([4, 3, 224, 224])
labels torch.Size([4])


In [ ]:
##################  Training settings ################
def train_model():
    model = ViTForImageClassification.from_pretrained(
        model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
    model.to(device)

    # Create specific directories for this combination of lr_scheduler_type and weight_decay
    var1 = "CollabA100"
    output_dir = experiment_dir / var1  / "output-models"
    log_dir = experiment_dir / var1  / "logs"
    emission_dir = experiment_dir / var1

    # Setup the emissions tracker with the experiment name
    tracker = EmissionsTracker(output_dir=emission_dir, output_file=f"emissions_{var1}.csv")

    # Capture the start time for training
    start_time = time.time()

    tracker.start()

    output_dir.mkdir(parents=True, exist_ok=True)
    log_dir.mkdir(parents=True, exist_ok=True)

    train_args = TrainingArguments(
        output_dir=str(output_dir),
        do_train=do_train,
        do_eval=do_eval,
        do_predict=do_predict,
        save_total_limit=2,
        report_to='tensorboard',
        save_strategy="epoch",
        eval_strategy="epoch",
        learning_rate=1e-5, #Freeze hyperparameter 1
        lr_scheduler_type='cosine', # freezed hyperparameter 2
        per_device_train_batch_size=16, # freezed hyperparameter 3
        per_device_eval_batch_size=4,
        load_best_model_at_end=True,
        logging_dir=str(log_dir),
        remove_unused_columns=False,
        logging_strategy="epoch",
        logging_steps=10,
        seed=seed,
        num_train_epochs=60, #fixed hyper parameter 4
        max_grad_norm=0.98
    )

    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=processor,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    # Save the final model
    model.save_pretrained(output_dir)
    processor.save_pretrained(output_dir)

    # Capture the end time
    end_time = time.time()
    emissions = tracker.stop()
    print(f"Emissions  for senario {name_of_experiment} : {emissions:.5f} kg CO2eq")

    # only for when eabling also run the test dataset predictions
    if do_predict:
        evaluate_and_log_metrics(trainer, test_ds, output_dir, processor)

In [ ]:
#################  Code Execution  ################
# Run training for each combination of learning rate scheduler type and weight decay sequentially
train_model()


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[codecarbon INFO @ 17:57:10] [setup] RAM Tracking...
[codecarbon INFO @ 17:57:10] [setup] GPU Tracking...
[codecarbon INFO @ 17:57:10] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:57:10] [setup] CPU Tracking...
[codecarbon WARNING @ 17:57:10] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:57:11] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 17:57:

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.833700,0.613578,0.714286,0.739613,0.714286,0.723758
2,0.611600,0.505631,0.809524,0.846172,0.809524,0.819103
3,0.546700,0.326276,0.904762,0.909524,0.904762,0.903662
4,0.456300,0.439285,0.825397,0.873457,0.825397,0.829972
5,0.376800,0.265632,0.920635,0.921737,0.920635,0.920859
6,0.438800,0.705362,0.730159,0.780693,0.730159,0.682171
7,0.367600,0.489102,0.809524,0.843331,0.809524,0.780776
8,0.339900,0.353060,0.841270,0.846593,0.841270,0.835140
9,0.326900,0.247031,0.888889,0.888889,0.888889,0.888889
10,0.303700,0.317946,0.888889,0.906704,0.888889,0.893065


[codecarbon INFO @ 17:57:19] Energy consumed for RAM : 0.000913 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 17:57:19] Energy consumed for all GPUs : 0.001743 kWh. Total GPU Power : 123.98081993890048 W
[codecarbon INFO @ 17:57:19] Energy consumed for all CPUs : 0.001240 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:57:19] 0.003895 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:57:26] Energy consumed for RAM : 0.000130 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 17:57:26] Energy consumed for all GPUs : 0.000921 kWh. Total GPU Power : 221.07636265997547 W
[codecarbon INFO @ 17:57:26] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:57:26] 0.001229 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:57:27] Energy consumed for RAM : 0.001827 kWh. RAM Power : 31.30389261245728 W
[codecarbon INFO @ 17:57:27] Energy consumed for all GPUs : 0.003589 kWh. Total GPU Power : 223.83329619731

Emissions  for senario CollabA100 : 0.06260 kg CO2eq
